In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, query_engine, StorageContext, load_index_from_storage
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.vector_stores.chroma import ChromaVectorStore

from pathlib import Path
from qdrant_client import QdrantClient, models
import chromadb



c:\Users\Ds-Terminal\Desktop\rag-llamaindex-chroma\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


In [2]:
db_path = "C:/Users/khalil/Desktop/fastapi_llamaindex/vector_stores"

In [3]:
llm = Ollama(
    base_url="http://192.168.2.62:11434/",
    model = "llama3:latest"
)

In [4]:
response = llm.complete("hello")
print(response)

Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?


In [5]:
embed_model = OllamaEmbedding(
    base_url="http://192.168.2.62:11434/",
    model_name = "nomic-embed-text:latest"
)
embeddings = embed_model.get_text_embedding("hello")
print(len(embeddings))

768


In [9]:
from core.settings import CHROMA_PATH

In [8]:
from llama_index.core import settings
settings.llm = llm
settings.embed_model = embed_model

In [10]:
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
chroma_client.__dict__

{'_identifier': 'C:\\Users\\Ds-Terminal\\Desktop\\rag-llamaindex-chroma\\vector_stores',
 'tenant': 'default_tenant',
 'database': 'default_database',
 '_server': <chromadb.api.rust.RustBindingsAPI at 0x217cd95bd90>,
 '_admin_client': <chromadb.api.client.AdminClient at 0x217cd95b890>}

In [12]:
chroma_client.list_collections()

[Collection(name=string), Collection(name=stringg)]

In [15]:
chroma_client.get_collection("stringg").__dict__

{'_client': <chromadb.api.rust.RustBindingsAPI at 0x217cd95bd90>,
 '_model': Collection(id=UUID('43417482-b4b8-4abc-aca9-c882f25be1cd'), name='stringg', configuration_json={'hnsw': {'space': 'cosine', 'ef_construction': 100, 'ef_search': 100, 'max_neighbors': 16, 'resize_factor': 1.2, 'sync_threshold': 1000}, 'spann': None, 'embedding_function': {'type': 'known', 'name': 'ollama', 'config': {'model_name': 'nomic-embed-text:latest', 'timeout': 60, 'url': 'http://192.168.2.62:11434'}}}, metadata=None, dimension=768, tenant='default_tenant', database='default_database', version=0, log_position=0),
 '_embedding_function': <chromadb.utils.embedding_functions.DefaultEmbeddingFunction at 0x217cd436f50>,
 '_data_loader': None}

In [ ]:

# vector_client = QdrantClient(":memory:")

vector_client = QdrantClient(path=db_path)  # Persists changes to disk
vector_client.__dict__

In [61]:

#Load documents from a folder
user_dir = "C:\\Users\\khalil\\Desktop\\fastapi_llamaindex\\data\\docs"
docs = SimpleDirectoryReader(user_dir).load_data()
print(f"Loaded {len(docs)} documents")

user_name = "khalil"


Loaded 1 documents


In [11]:
docs[0]

NameError: name 'docs' is not defined

In [64]:
try:
    vector_client.get_collection(
    collection_name=user_name,
    vectors_config={
        "text-dense":models.VectorParams(
        size=768,
        distance=models.Distance.COSINE,
    )}
)
    print(f"Collection '{user_name}' exists")
except Exception:
    vector_client.create_collection(
    collection_name=user_name,
    vectors_config={
        "text-dense":models.VectorParams(
        size=768,
        distance=models.Distance.COSINE,
    )}
)
    print(f"Created new collection '{user_name}'")



Created new collection 'khalil'


In [65]:
collection = vector_client.get_collection(user_name)
collection.__dict__


{'status': <CollectionStatus.GREEN: 'green'>,
 'optimizer_status': <OptimizersStatusOneOf.OK: 'ok'>,
 'vectors_count': None,
 'indexed_vectors_count': 0,
 'points_count': 0,
 'segments_count': 1,
 'config': CollectionConfig(params=CollectionParams(vectors={'text-dense': VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_m

In [66]:
vector_store = QdrantVectorStore(collection_name=user_name,client=vector_client)
vector_store.__dict__

{'stores_text': True,
 'is_embedding_query': True,
 'flat_metadata': False,
 'collection_name': 'khalil',
 'url': None,
 'api_key': None,
 'batch_size': 64,
 'parallel': 1,
 'max_retries': 3,
 'client_kwargs': {},
 'enable_hybrid': False,
 'index_doc_id': True,
 'fastembed_sparse_model': None,
 'text_key': 'text',
 'dense_vector_name': 'text-dense',
 'sparse_vector_name': 'text-sparse-new',
 '_user_provided_sparse_doc_fn': False,
 '_user_provided_sparse_query_fn': False}

In [67]:

# 6️⃣ Create storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)
storage_context.__dict__




{'docstore': <llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore at 0x201c5d9f100>,
 'index_store': <llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore at 0x201c5d9f230>,
 'vector_stores': {'default': QdrantVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=False, collection_name='khalil', url=None, api_key=None, batch_size=64, parallel=1, max_retries=3, client_kwargs={}, enable_hybrid=False, index_doc_id=True, fastembed_sparse_model=None, text_key='text', dense_vector_name='text-dense', sparse_vector_name='text-sparse-new'),
  'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))},
 'graph_store': <llama_index.core.graph_stores.simple.SimpleGraphStore at 0x201c5d9efd0>,
 'property_graph_store': None}

In [66]:
index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store
    )
print("Index loaded from existing vector store!")

Index loaded from existing vector store!


In [68]:
vector_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='string'), CollectionDescription(name='stringg'), CollectionDescription(name='khalil')])

In [69]:
index = VectorStoreIndex.from_documents(documents=docs, storage_context=storage_context, embed_model = embed_model)

2025-10-08 13:02:36,313 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"


In [71]:
index_after_saving = load_index_from_storage(storage_context=storage_context)
print("Index loaded from existing vector store!")

2025-10-08 13:04:45,736 - INFO - Loading all indices.


Index loaded from existing vector store!


In [73]:
index_after_saving.__dict__

{'_use_async': False,
 '_store_nodes_override': False,
 '_embed_model': OllamaEmbedding(model_name='nomic-embed-text:latest', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x00000201C236D090>, num_workers=None, embeddings_cache=None, base_url='http://192.168.2.62:11434', ollama_additional_kwargs={}, query_instruction=None, text_instruction=None),
 '_insert_batch_size': 2048,
 '_storage_context': StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x00000201C5D9F100>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x00000201C5D9F230>, vector_stores={'default': QdrantVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=False, collection_name='khalil', url=None, api_key=None, batch_size=64, parallel=1, max_retries=3, client_kwargs={}, enable_hybrid=False, index_doc_id=True, fastembed_sparse_model=None, text_key='text', den

In [74]:
engine = index_after_saving.as_query_engine(llm = llm)
result = engine.query("turbo").response
print(result)

2025-10-08 13:05:24,244 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/embed "HTTP/1.1 200 OK"


2025-10-08 13:05:29,207 - INFO - HTTP Request: POST http://192.168.2.62:11434/api/chat "HTTP/1.1 200 OK"


GPT-3.5 Turbo.


In [6]:
# ===============================================================
# LlamaIndex + Ollama + Persistent Chroma (Correct Schema Version)
# ===============================================================

from llama_index.core import VectorStoreIndex, StorageContext, Document
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.ollama import OllamaEmbedding
import chromadb

# ---------------------------------------------------------------
# 1. Create or connect to a persistent Chroma database
# ---------------------------------------------------------------
chroma_path = "C:/Users/khalil/Desktop/fastapi_llamaindex/vector_stores"
client = chromadb.PersistentClient(path=chroma_path)

# ---------------------------------------------------------------
# 2. Define the Chroma collection schema BEFORE inserting data
# ---------------------------------------------------------------
collection_name = "ollama_docs"

# Delete existing collection (optional during development)
try:
    client.delete_collection(name=collection_name)
except Exception:
    pass

# Create collection with metadata (schema hint)
collection = client.create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}  # cosine distance for embeddings
)

# ---------------------------------------------------------------
# 3. Wrap Chroma collection with LlamaIndex VectorStore
# ---------------------------------------------------------------
vector_store = ChromaVectorStore(chroma_collection=collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

# ---------------------------------------------------------------
# 4. Define the Ollama embedding model
# ---------------------------------------------------------------
embed_model = OllamaEmbedding(
    model_name="nomic-embed-text",          # model name in Ollama
    base_url="http://192.168.2.62:11434"       # your Ollama API endpoint
)

# ---------------------------------------------------------------
# 5. Create sample documents
# ---------------------------------------------------------------
docs = [
    Document(text="Llamas are domesticated South American camelids, known for their wool."),
    Document(text="The Andes mountains are home to many unique animals."),
    Document(text="Machine learning enables computers to learn from data.")
]





In [2]:
embed_model.get_text_embedding("hello")

[0.017934207,
 -0.0058618574,
 -0.1753405,
 -0.0137597285,
 0.034045048,
 0.044717424,
 0.012389901,
 -0.002573614,
 -0.014818539,
 -0.03926486,
 -0.009212158,
 0.051710434,
 0.05772606,
 0.05718845,
 0.045006063,
 -0.053577494,
 0.02879359,
 -0.04719623,
 -0.039045226,
 0.026926365,
 0.009360123,
 -0.06674427,
 0.00494737,
 -0.0058786357,
 0.17223418,
 -0.0044028154,
 0.018685732,
 0.08336984,
 0.0016030574,
 -0.022514228,
 0.020344356,
 -0.018736571,
 0.015327567,
 0.0031978537,
 0.027555587,
 -0.018736059,
 -0.003822045,
 0.011286019,
 0.016174829,
 0.019278742,
 -0.01763444,
 -0.009617367,
 0.009247078,
 -0.028306274,
 0.051377025,
 0.007286372,
 -0.02211025,
 0.013347005,
 0.053673193,
 -0.043458115,
 -0.0040050703,
 -0.0041706553,
 -0.022164641,
 0.04238723,
 0.033484377,
 0.05498299,
 0.057873204,
 0.020419374,
 0.021775909,
 0.025710681,
 0.0377922,
 0.047355488,
 0.019401494,
 0.07026736,
 -0.002652634,
 -0.032029673,
 -0.010793113,
 0.037769657,
 0.047647636,
 -0.0041703577,


In [ ]:
from llama_index.core import Settings
Settings.embed_model = embed_model
settings.__dict__

In [20]:
# ---------------------------------------------------------------
# 6. Build and persist the index
# ---------------------------------------------------------------
index = VectorStoreIndex.from_documents(
    docs,
    storage_context=storage_context
)

print("✅ Index built and stored in:", chroma_path)

: 

In [7]:
collection = client.get_collection(collection_name)
collection.peek()

{'ids': [],
 'embeddings': array([], dtype=float64),
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings'],
 'data': None,
 'metadatas': []}

In [ ]:

# ---------------------------------------------------------------
# 7. Reopen the persistent index later (simulating a new session)
# ---------------------------------------------------------------
client = chromadb.PersistentClient(path=chroma_path)
collection = client.get_collection(collection_name)
vector_store = ChromaVectorStore(chroma_collection=collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Recreate index from the existing vector store
index = VectorStoreIndex.from_vector_store(
    vector_store,
    storage_context=storage_context,
    embed_model=embed_model
)

# ---------------------------------------------------------------
# 8. Query the index
# ---------------------------------------------------------------
query_engine = index.as_query_engine()
query = "What animal is known for its wool?"
response = query_engine.query(query)

print("\n🔍 Query:", query)
print("💬 Answer:", response)